# Machine Learning Fairness

_**Note**:  the goal of this part of the assignment is to understand the kinds of biases that commonly sneak into machine learned systems and a handful of techniques to improve standard modeling.  While we hope you find this instructive, we empathize that these research results may negatively affect some students.  Please reach out to the teaching staff if you have serious concerns for alternate arrangements._

From simple count-based models to the most complex neural architectures, machine learning models are ultimately nothing more than the product of the signals and labels in the training set.  That these tools can so effectively mimic and generalize from the training set distribution is the key to why they are so useful in so many applications.

This powerful ability to fit a data is a double edged sword.  Unfortunately, the real world is filled with inequality, unfairness and stereotypes.  When the signals and labels systemically capture these aspects of the world, the powerful ability to generalize has other names: bias.  This bias can take many forms:  a minority group of entries in the training set would be underrepresented (the loss function is incented to produce a model that works better on the majority at the expense of the minority) or predictions may be systemically biased against a protected group (i.e. the model learns to predict the protected label and from that the actual prediction rather than learning the prediction directly).

In this part of the assignment, we will take a look at a few nice analyses that discuss this bias. Below are a few questions about these papers.

- [How to make a racist AI without really trying](http://blog.conceptnet.io/posts/2017/how-to-make-a-racist-ai-without-really-trying/)
- [Man is to Computer Programmer as Woman is to Homemaker? Debiasing Word Embeddings](https://arxiv.org/pdf/1607.06520.pdf)
- [Data Decisions and Theoretical Implications when Adversarially Learning Fair Representations](https://arxiv.org/pdf/1707.00075.pdf)

**Again, please be sure to put your answers in the "answers" file!**

## Questions about the Racist AI

1.  In [Step 5](http://blog.conceptnet.io/posts/2017/how-to-make-a-racist-ai-without-really-trying/#Step-5:-Behold-the-monstrosity-that-we-have-created), the author shows that substituting a type of cuisine into a fixed sentence significantly changes the overall sentiment score of their model.  What is the difference in sentiment score between the word ```Italian``` and ```Mexican``` (not the difference in the whole sentence!), assuming that embeddings for all words in the sentence are found in GloVe.?

2. Rank ConceptNet Numberbatch, GloVe and Word2Vec by ethnic bias as defined by the author?

3. What technique does the author apply to achieve that lower bias?

4. How significant is the penalty to model performance when using debiased vectors instead of biased ones?

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import seaborn
import re
import statsmodels.formula.api

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# Configure how graphs will show up in this notebook
%matplotlib inline
seaborn.set_context('notebook', rc={'figure.figsize': (10, 6)}, font_scale=1.5)

In [3]:
def load_embeddings(filename):
    """
    Load a DataFrame from the generalized text format used by word2vec, GloVe,
    fastText, and ConceptNet Numberbatch. The main point where they differ is
    whether there is an initial line with the dimensions of the matrix.
    """
    labels = []
    rows = []
    with open(filename, encoding='utf-8') as infile:
        for i, line in enumerate(infile):
            items = line.rstrip().split(' ')
            if len(items) == 2:
                # This is a header row giving the shape of the matrix
                continue
            labels.append(items[0])
            values = np.array([float(x) for x in items[1:]], 'f')
            rows.append(values)
    
    arr = np.vstack(rows)
    return pd.DataFrame(arr, index=labels, dtype='f')

embeddings = load_embeddings('data/glove.42B.300d.txt')
embeddings.shape

FileNotFoundError: [Errno 2] No such file or directory: 'data/glove.42B.300d.txt'

## Questions about Debiasing Word Embeddings

Word embeddings are commonly used in deep neural networks to solve analogy tasks (see the Embddings worksheet, the corresponding sections in both [Word2Vec](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) and [GloVe](https://nlp.stanford.edu/pubs/glove.pdf)).  This paper quickly reintroduces that task, then continues to explore the analogy task with additional tuples that illustrate the bias that these vectors have picked up.

1.  Why are the results of Table 1 important?

### Questions about Adversarial Learning

1.  What is the intuition behind the parity gap measure?
2.  What is the intuition behind the equality gap measure?
3.  What is the intuition behind $J_{\lambda}$?